Solve the last example empirically. Let's simulate a chain and empirically compute the expected times 

In [2]:
import numpy as np

# Transition matrix from the example that we've solved
P = np.array([
    [0.5, 0.3, 0.2],
    [0.1, 0.6, 0.3],
    [0.0, 0.4, 0.6]
])

In [7]:
# import numpy as np

# # Transition matrix from the example that we've solved
# P = np.array([
#     [0.5, 0.3, 0.2],
#     [0.1, 0.6, 0.3],
#     [0.0, 0.4, 0.6]
# ])

target_state = 2
total_attempts = []
for _ in range(10000):
    prev_state = 0
    attempts = 0
    for _ in range(30):
        cur_state = np.random.choice([0,1,2], p=P[prev_state,:])
        attempts += 1
        prev_state = cur_state
        if cur_state == target_state:
            total_attempts.append(attempts)
            break
    

np.mean(total_attempts)





# # Simulate a trajectory and return the time to hit target state.
# def simulate_hitting_time(start_state, target_state=2, P=P, n=10000):

#     # define the simulation length
#     state = start_state
#     t = 0
#     # start the loop
#     while t < n:
#         if state == target_state:
#             return t
#         # we have 3 states: 0, 1, 2
#         state = np.random.choice([0,1,2], p=P[state])
#         t += 1
#     return 

# # now repeat this simulation for a number of times nad compute the mean
# def estimate_expected_time(start_state, trials=10000):
#     times = []
#     for _ in range(trials):
#         ht = simulate_hitting_time(start_state)
#         times.append(ht)
#     return np.mean(times)



np.float64(4.119123824764953)

In [6]:
#  RUN SIMULATION 

trials = 20000

est_h1 = estimate_expected_time(start_state=0, trials=trials)
est_h2 = estimate_expected_time(start_state=1, trials=trials)

print(f"Estimated E[T3, start at 1]: {est_h1:.4f}", ', analytical 4.1176')
print(f"Estimated E[T3, start at 2]: {est_h2:.4f}", ', analytical 3.5294')

Estimated E[T3, start at 1]: 4.1181 , analytical 4.1176
Estimated E[T3, start at 2]: 3.5202 , analytical 3.5294


Markov chains: some work with text!

In [8]:
#import numpy as np

# first, read the text file
# change encoding to utf-8-sig, otherwise it will add a Byte order mark (BOM, a unicode symbol) in the beginning of the file!
with open('pride_and_prejudice.txt',mode='r',  encoding='utf-8-sig') as f:
    book = f.read()
    

FileNotFoundError: [Errno 2] No such file or directory: 'pride_and_prejudice.txt'

In [9]:
from collections import OrderedDict
import itertools

# let's remove some the text in the beginning and leave only the actual text of the book. 
normal_words = book.split()[105:]

# add dots
for i in range(len(normal_words)):
    if '.' in normal_words[i] and len(normal_words[i]) > 1:
        normal_words[i] = normal_words[i].replace(".", "")
        normal_words.insert(i+1, '.')

# and also remove some text in the end
normal_words = normal_words[:len(normal_words)-2926]

print(normal_words[len(normal_words)-10:len(normal_words)])


# define the vector of states (unique words)
# use this and not SET to preserve the order! 
states = list(OrderedDict.fromkeys(normal_words).keys())

print(len(normal_words))
print(len(states))


NameError: name 'book' is not defined

Create a dictionary where keys are the actual words and values are the integer labels

In [10]:
import numpy as np
# enumrate the states
labs = np.arange(0, len(states))

# turn the sequence of words into a sequence of integers, and map them together into a dictionary
res = {states[i]: labs[i] for i in range(len(labs))}

print(res.values())


NameError: name 'states' is not defined

Now make a method for mapping the words into integers and back

In [10]:
def map_words_into_integers(normal_words):
    mapped_words = []

    for cur_word in normal_words:
        cur_ind = list(res.keys()).index(cur_word)
        mapped_words.append(list(res.values())[cur_ind])
        
    return mapped_words


In [11]:
def map_integers_into_words(integers):
    real_words = []

    for cur_word in integers:
        cur_ind = list(res.values()).index(cur_word)
        real_words.append(list(res.keys())[cur_ind])
        
    return real_words

and define a method to create transition matrix

In [14]:
def transition_matrix(chain_seq):
    # define number of states
    n = 1+ max(chain_seq) 
    # create an empty nXn matrix, which we will fill in
    M = np.zeros((n,n))
    # zip is a useful method for working with matrices
    # for each states, we add up to a matrix whenever we find this state in the sequence 
    for (i,j) in zip(chain_seq,chain_seq[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    M = M/M.sum(axis=1, keepdims=True)
    return M

test the function: first, make integers out of words (might take a while)

In [12]:
# test the function: first, make integers out of words (might take a while)

words_mapped = map_words_into_integers(normal_words)



Now compute the transition matrix

In [15]:
p_words = transition_matrix(words_mapped)

# check the sum across rows, should be one!
p_words.sum(axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

Compute the probabilty of going form state "the" to "her"

In [17]:
ind1 = list(res.keys()).index('the')
ind2 = list(res.keys()).index('her')

print(ind1)
print(ind2)

print(states[ind1])
print(states[ind2])

# the probability of going from THE to HER we take directly from the matrix 
print('Probability of going from state THE to state HER is ', p_words[ind1, ind2])

24
237
the
her
Probability of going from state THE to state HER is  0.0


What about "His" and "wife"?

In [19]:
ind_his = list(res.keys()).index('his')
ind_wife = list(res.keys()).index('wife')

p_words[ind_his, ind_wife]

print('Probability of going from state HIS to state WIFE is ', p_words[ind1, ind2])

Probability of going from state HIS to state WIFE is  0.0


Can also estimate the expected number of steps required to go from "the" to "her"

In [ ]:
ind1 = list(res.keys()).index('the')
ind2 = list(res.keys()).index('her')

## first, find all indices of "her" and "the" in the sequence
starts = [i for i, x in enumerate(words_mapped) if x == ind1]
ends   = [i for i, x in enumerate(words_mapped) if x ==ind2]

## now take the length of ALL subsets WITH these indicies
#all_subsets_required_states = [len(words_mapped[start:end+1]) for start,end in zip(starts, ends)]
########3 this does not keep the order! Try this:

# define a function that takes a vector of integers, start index and end index, and finds all
# subsequence between these indices 
def get_length_all_sequences(seq, ind1, ind2):
    # set a count of steps
    count = 0
    # set a flag
    fl = 0
    # keep the length of all sequences between "The" and "Her"
    sequence_length = []
    for i in seq:
        if i == ind1:
            fl = 1
        elif i == ind2:
            fl = 0
            sequence_length.append(count)
            count = 0
        # if flag is on, i count words
        if fl: 
            count+= 1
    return sequence_length


In [58]:
ind1 = list(res.keys()).index('his')
ind2 = list(res.keys()).index('wife')

# and compute the mean
sequence_length = get_length_all_sequences(words_mapped,ind1, ind2)
np.mean(sequence_length)


4529.444444444444

Some fun with text: Generate samplings of your own text using the estimated probability matrix! 

In [33]:
from collections import Counter

# function that generates text
def generate_text(states, p_words, init_state, N_words):

        # generate some text with N words

        # for the current word, generate the new word k times and choose the most common one
        k = 100
        N = N_words

        n_states = len(states)
        # p_words is the matrix

        # initial state:
        i1 = init_state
        all_words = [i1]
        for i in range(1, N):
            # all words generated k times for the current state
            cur_states_word = []
            # given the current state, choose the column of the matrix accordingly 
            p_column = p_words[all_words[i-1]]
            # generate k words according to the matrix
            for j in range(k):
                cur_states_word.append(np.random.choice(np.arange(0, n_states), p = p_column))
                
            # now choose the most common word
            most_common_word = list(Counter(cur_states_word).most_common(1)[0])[0]
            all_words.append(most_common_word)
            
        return all_words

def generate_and_write(p_words, init_state, filename, N_words):


    text_integers = generate_text(states, p_words, init_state, N_words)
    my_text = map_integers_into_words(text_integers)

    whole_text = ' '.join(my_text)

    with open(filename, "w") as text_file:
        text_file.write(whole_text)
        
    return whole_text

In [53]:
p_my_book = p_words

p_my_book[np.isnan(p_my_book)] = 0


in_state = ind2 = list(res.keys()).index('She')

text = generate_and_write(p_my_book, in_state, 'some-text-new-try.txt', 15)
text


'She was very great deal of her . I am sure you must be in'

Markov chain examples: Stationary distribuiton. Recall: pi*M = pi

In [4]:
import numpy as np
# define some matrix of probabilities
transition_matrix = np.array([[0,0.5,0.5],
                              [0,0.3,0.7],
                              [1,0,0]])

# do the rows sum to 1?
print(transition_matrix.sum(axis=1))

transition_matrix


[1. 1. 1.]


array([[0. , 0.5, 0.5],
       [0. , 0.3, 0.7],
       [1. , 0. , 0. ]])

 No built-in method to find stationary distribution. One of the algorithms:
 Find the eigenvector with an eigenvalue that is are close to one. To compute the stationary distribution, every element in the selected eigenvector is divided by the sum of the eigenvector

In [6]:
# transpose the matrix
transition_matrix_t = transition_matrix.T
# use linear algebra module of numpy to compute the eigenvalues
eigenvals, eigenvects = np.linalg.eig(transition_matrix_t)

# Find the indexes of the eigenvalues that are close to one.
close_to_1_idx = np.isclose(eigenvals,1) # isclose checks element-wise if each element is close to 1 within the specific tolerance
# Use them to select the target eigen vectors
target_eigenvect = eigenvects[:,close_to_1_idx]
#Flatten
target_eigenvect = target_eigenvect[:,0]
# Turn the eigenvector elements into probabilities
# that would be the stationary distribution
st_d = target_eigenvect / sum(target_eigenvect) 

In [9]:
# get the real part of the probability
print(st_d.real)

# our stationary distribution pi
real_st_distr = st_d.real

# check if they sum to 1
st_d.real.sum()

[0.36842105 0.26315789 0.36842105]


1.0

Check the fixed point equation: pi*M = pi

In [10]:
real_st_distr.dot(transition_matrix)


array([0.36842105, 0.26315789, 0.36842105])